In [15]:
# default_exp encryption

# Encryption

> Functions for encrypting files.

In [16]:
# export
import cryptography
from cryptography.fernet import Fernet
from fastcore.all import *
from pathlib import Path
from functools import partial
from shutil import copyfile
import os

In [17]:
#hide
from nbdev.showdoc import *

## Utility Functions

In [18]:
# export
def process_dirs(p:Path,fn=None,invalid_exts='mca'):
    p=Path(p)
    if p.is_file(): return fn(p,invalid_exts=invalid_exts)
    for dirpath,dirnames,filenames in os.walk(str(p)):
        if len(dirnames)==0 and len(filenames)==0: continue
        for f in filenames: fn(Path(dirpath)/Path(f),invalid_exts=invalid_exts)

In [19]:
process_dirs('test_save',lambda x,invalid_exts:print(x))

test_save/level.dat
test_save/level.dat_old
test_save/session.lock
test_save/multipart.dat
test_save/forcedchunks.dat
test_save/advancements/aaa58dde-8e1d-4577-aae3-fff207e8c068.json
test_save/playerdata/aaa58dde-8e1d-4577-aae3-fff207e8c068.dat
test_save/DIM-17/multipart.dat
test_save/DIM-17/forcedchunks.dat
test_save/DIM-17/data/buildcraft_wire_systems.dat
test_save/DIM-17/data/capabilities.dat
test_save/DIM-17/data/villages_wyvern_lair.dat
test_save/DIM-1/multipart.dat
test_save/DIM-1/forcedchunks.dat
test_save/DIM-1/data/buildcraft_wire_systems.dat
test_save/DIM-1/data/villages_nether.dat
test_save/DIM-1/data/capabilities.dat
test_save/minecolonies/colonies.dat
test_save/minecolonies/colonies-2020-12-07_23.04.22.zip
test_save/data/villages.dat
test_save/data/buildcraft_wire_systems.dat
test_save/data/buildcraft_marker_volume.dat
test_save/data/Village.dat
test_save/data/mocreatures.dat
test_save/data/capabilities.dat
test_save/data/buildcraft_marker_path.dat
test_save/data/Mineshaft

## Crypto Functions

In [20]:
%%writefile test_file.txt
Hello
This is
A file that 
needs to be encrypted

Overwriting test_file.txt


In [21]:
# export
pv=lambda msg,v:(print(msg) if v else None)

def encrypt_file(p:Path,to_p:Path=None,block_sz=65536,verbose=False,prefix='encrypted',key=None,
                invalid_exts=''):
    p=Path(p)
    to_p=ifnone(to_p,Path(prefix)/p)        
    key=ifnone(key,Fernet.generate_key())
    encryptor=Fernet(key)
    os.makedirs(str(Path(to_p).parent),exist_ok=True)
    
    if p.suffix[1:] in invalid_exts.split(','): 
        copyfile(str(p),str(to_p))
        return key 
    
    with open(p,'rb') as f,open(to_p,'wb') as to_f:
        fb=f.read(block_sz)
        while len(fb)>0: 
            token=encryptor.encrypt(fb)
            pv(token,verbose)
            to_f.write(token)
            fb=f.read(block_sz) 
    return key

In [22]:
k=encrypt_file('test_file.txt',block_sz=8,verbose=True);k

b'gAAAAABf1AeT-YBxb0mutyFSHsyRDPkg1ymqmiPs96HNeV8TdQiUErqv29ujqK8Pvd0Eg-DF0qG0fUfHnZhHCG37s6MVEI9vXA=='
b'gAAAAABf1AeTjOKgAo9W6qWmXfYZmEmIpsNh8hUBRv5rlhFVCNtsQN_ac7xs7AJOXRcpfm0fhW5Nyj4-ZbuWXT07ihj9nQCvWQ=='
b'gAAAAABf1AeTLbXmex_EYUJJ75ZwqqZmCxdnvrB99EW6KO46lC19ioIlTWxttZ0A-Ld14-5bkE--pCkkQG4HGppEhvpTG29arw=='
b'gAAAAABf1AeTmd9Xp_HG56EN56dnijcDggswbmg7hjO34BYFCh8CvF09kay0bsUQ4ajIvakiU-HCvOZuVPzdRE2nByixqehQTA=='
b'gAAAAABf1AeTajYSfpV2JMR4MqiZCisfqQZKtf0Nj8aFXEHJPteSyqoFMjmz61SovxhLbWc9cBNPlhxyTFRe65dGaZJ5eya6jA=='
b'gAAAAABf1AeTDCe8FblH-K-tUOFQJVePCf9u4OzoGbC7txeszeC3psHhX7q80GqqvIEPCzxVFqlRne6GlZ8LXo0nIZITsCw4tA=='
b'gAAAAABf1AeTN28N4WaS_cZxMDzXvusIQtwV6yxiFJCe1-RoeottPrRY8JisOruIKAku5B06rz1Memmzpk_z45zAx4NTYytFyw=='


b'8r5mrM7RtDYgJNbWkv62LoXMuj_AROATx_GZdH1WAcs='

In [23]:
# export
def decrypt_file(p:Path,key,to_p:Path=None,block_sz=65536,verbose=False,prefix='decrypted',
                invalid_exts=''):
    p=Path(p)
    pv(p,verbose)
    to_p=ifnone(to_p,p.parents[2]/prefix/p if len(p.parts)>2 else prefix/p)
    decryptor=Fernet(key)
    os.makedirs(str(Path(to_p).parent),exist_ok=True)
    if p.suffix[1:] in invalid_exts.split(','): 
        copyfile(str(p),str(to_p))
        return key 
    
    with open(p,'rb') as f,open(to_p,'wb') as to_f:
        fb=f.read(block_sz)
        while len(fb)>0: 
            token=decryptor.decrypt(fb)
            to_f.write(token)
            fb=f.read(block_sz) 
    return key

In [24]:
decrypt_file('encrypted/test_file.txt',k,'test_file_decrypted.txt',verbose=True)

encrypted/test_file.txt


b'8r5mrM7RtDYgJNbWkv62LoXMuj_AROATx_GZdH1WAcs='

In [25]:
k=Fernet.generate_key()
process_dirs('test_save',partial(encrypt_file,key=k,prefix='minecraft_encrypted'))

In [26]:
process_dirs('minecraft_encrypted/test_save',partial(decrypt_file,key=k,verbose=True))

minecraft_encrypted/test_save/level.dat
minecraft_encrypted/test_save/level.dat_old
minecraft_encrypted/test_save/session.lock
minecraft_encrypted/test_save/multipart.dat
minecraft_encrypted/test_save/forcedchunks.dat
minecraft_encrypted/test_save/advancements/aaa58dde-8e1d-4577-aae3-fff207e8c068.json
minecraft_encrypted/test_save/playerdata/aaa58dde-8e1d-4577-aae3-fff207e8c068.dat
minecraft_encrypted/test_save/DIM-17/multipart.dat
minecraft_encrypted/test_save/DIM-17/forcedchunks.dat
minecraft_encrypted/test_save/DIM-17/data/buildcraft_wire_systems.dat
minecraft_encrypted/test_save/DIM-17/data/capabilities.dat
minecraft_encrypted/test_save/DIM-17/data/villages_wyvern_lair.dat
minecraft_encrypted/test_save/DIM-1/multipart.dat
minecraft_encrypted/test_save/DIM-1/forcedchunks.dat
minecraft_encrypted/test_save/DIM-1/data/buildcraft_wire_systems.dat
minecraft_encrypted/test_save/DIM-1/data/villages_nether.dat
minecraft_encrypted/test_save/DIM-1/data/capabilities.dat
minecraft_encrypted/tes

In [27]:
# hide
from nbdev.export import *
from nbdev.export2html import *
from nbdev.cli import make_readme
make_readme()
notebook2script()
notebook2html()

converting /opt/project/play_it_later/nbs/index.ipynb to README.md
Converted 00_encryption.ipynb.
Converted 01_talker.ipynb.
Converted 02_cli.ipynb.
Converted index.ipynb.
converting: /opt/project/play_it_later/nbs/00_encryption.ipynb
converting: /opt/project/play_it_later/nbs/02_cli.ipynb
An error occurred while executing the following cell:
------------------
from nbdev.showdoc import show_doc
from play_it_later.cli import *
------------------

Traceback (most recent call last):

  File "/opt/conda/envs/play_it_later/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)

  File "<ipython-input-1-874cf4e0a969>", line 2, in <module>
    from play_it_later.cli import *

  File "/opt/project/play_it_later/play_it_later/cli.py", line 56
    mv_original_dir:Param('Move to the original directory.',bool_arg)=False):
                  ^
SyntaxError: invalid syntax

SyntaxError: invalid syntax (cli.py, line 5